In [22]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.subplots as sp
import plotly.express as px
import numpy as np

In [24]:
n_colors = 10

colors = px.colors.sample_colorscale("turbo", [n/(n_colors -1) for n in range(n_colors)])

In [18]:
mid_domain = 24
node_data = [{
    'east': [[10, 30], [mid_domain, mid_domain]],
    'west': [[5, 20], [mid_domain, mid_domain]],
    'north': [[mid_domain, mid_domain], [30, 40]],
    'south': [[mid_domain, mid_domain], [10, 20]]
},{
    'east': [[10, 30], [mid_domain, mid_domain]],
    'west': [[5, 20], [mid_domain, mid_domain]],
    'north': [[mid_domain, mid_domain], [30, 40]],
    'south': [[mid_domain, mid_domain], [10, 20]]
},{
    'east': [[10, 30], [mid_domain, mid_domain]],
    'west': [[5, 20], [mid_domain, mid_domain]],
    'north': [[mid_domain, mid_domain], [30, 40]],
    'south': [[mid_domain, mid_domain], [10, 20]]
}]


In [26]:
fig = sp.make_subplots(rows=1, cols=3, subplot_titles=[1,2,3])

for ix, faces in enumerate(node_data):
    showlegendbool = True if ix == 0 else False
    for iy, (face_name, face_coords) in enumerate(faces.items()):
        fig.add_trace(go.Scatter(x=face_coords[0], y=face_coords[1], name=face_name, marker_color=colors[iy], showlegend=showlegendbool), row=1, col=ix+1)


fig.show()

In [9]:
# plot one faces data on number line -> made up 
mid_domain = 24
faces = {
    'east': [[10, 30], [mid_domain, mid_domain]],
    'west': [[5, 20], [mid_domain, mid_domain]],
    'north': [[mid_domain, mid_domain], [30, 40]],
    'south': [[mid_domain, mid_domain], [10, 20]]
}

fig = go.Figure()

for face_name, face_coords in faces.items():
    fig.add_trace(go.Scatter(x=face_coords[0], y=face_coords[1], name=face_name))

fig.show()